|        |        |        |
|--------|--------|--------|
![H-BRS](logos/h-brs.png) | ![A2S](logos/a2s.png) | ![b-it](logos/b-it.png) |

# Autonomous Mobile Robots

# AMR Assignment 5

### General information

* Please do not add or delete any cells. Answers belong into the already provided cells (below the question).
* If a function is given (either as a signature or a full function), you should not change the name, arguments, or return value of the function.
* If you encounter empty cells underneath the answer that can not be edited, please ignore them; they are for testing purposes.
* Please note that variables declared in the notebook cells have global scope. To make sure your assignment works correctly as a whole, please restart the kernel and run all cells before submitting (e.g. via *Kernel -> Restart & Run All*).
* Code cells where you are supposed to give your answer often include the line  ```raise NotImplementedError```. This makes it easier to automatically grade answers. Once you fill out a function, please delete this line.

### Submission

Please make sure to write all your team members 2s IDs in the cell below before submission. Please submit your notebook via the JupyterHub web interface (in the main view -> Assignments -> Submit). If it is a group assignment, please make only one submission per group (for easier bookkeeping, it is best if this is always the same team member).

### Questions about the assignment

If you have questions about the assignment, you are encouraged to post them in the LEA forum. Proactive discussions lead to better understanding. Let's keep the forum active.

## Team members (2s IDs):

* kabdul2s
* nravi2s
* ysirin2s

# Landmark-based Localisation using Kalman Filter (100 points)

In this assignment, we will use a Kalman filter for localisation in an environment based on the measurement of landmark locations. As you already know, sensor readings are affected by noise, making it difficult to achieve accurate measurement of the physical quantity under consideration. The Kalman filter, which belongs to the family of Bayes filter algorithms, can be used for minimising the error in the estimation. The belief of the state (in this case, it is the **pose of the robot** with respect to the **map frame**) is represented as a Gaussian distribution. The objective is to estimate the mean $\mu$ and covariance matrix $\Sigma$ for the physical quantity under consideration at every time instance.

In this assignment, the Robile is placed in a simulation environment in Gazebo, where several RFID tags are positioned on the ground. Now, suppose that a virtual RFID detector is mounted on the robot, sharing the *same frame* and *field of view* as the LiDAR. In the real world, the robot's real pose almost never matches with its ideal motion model due to different sources of noise, such as from the encoder readings, unstructured environment interaction, as well as other noisy perception data.

In this assignment, we will explore how the Kalman filter can be employed to improve the estimation of the robot's pose in this setup. Below is the simulation environment where the RFID tags are highlighted.

![robile_localisation_kalman](img/robile_localisation_kalman.png)

The positions of the RFID tags as $(x,y)$ coordinates in the `odom` frame with tag names are as follows:  
- A: $(1.0, 1.0)$
- B: $(6.0, 1.0)$
- C: $(3.0, -1.0)$
- D: $(1.0, -3.0)$
- E: $(4.0, -4.0)$

### Motion noise consideration

When the robot is moving in the environment, the noise in the data published onto the `/odom` topic is negligible in the simulation. To simulate the real-world noise, for every message on the `/odom` topic, some noise is introduced manually in the associated *robile_rfid_tag_finder.py* script, and the variable `real_base_link_pose` is updated accordingly. Here, the data on the `/odom` topic, and consequently the visualization of the robot model in RVIZ, can be considered as an ideal motion model, which is the motion estimate without noise; the data on `real_base_link_pose` is then the actual pose of the robot's base (`real_base_link` frame) with respect to which the RFID tags are measured, and which are available on the `/rfid_tag_poses` topic. The `real_base_link_pose` topic thus represents the actual pose of the robot, which deviates from `/odom` due to uncertain interaction with the environment.

Your goal is to read from the `odom` topic and, along with the measured RFID tag positions, estimate the posterior belief of the robot's pose and the variance associated with it. This estimated pose, which should be published to the `estimated_base_link_pose` topic, should eventually converge to the pose represented on `real_laser_link_pose`.

### Summary

- The pose of the robot with respect to the odom frame, published on the `/odom` topic represents ideal motion without noise.
- Since it is challenging to mathematically represent or model all physical interactions with the environment, in reality, the robot deviates from the ideal motion and its pose estimation is affected by noise in the sensor data and an incomplete model of the interaction with the environment. The actual robot pose is considered to be the `real_base_link` frame.
- The measurement model is the pose estimate using the detected RFID tags on the `/rfid_tag_poses` topic, which is represented by a custom message of type ``PoseLabelledArray``. These tags are measured with respect to the `real_base_link` frame.
- Note that the `real_base_link` frame is only used to visualise the actual robot pose; thus, only the data on `/odom` and `/rfid_tag_poses` topics should be used to determine robot's posterior pose `estimated_base_link_pose`.


### Notes

- The initial `real_base_link` frame is considered to be overlapping with the `odom` frame.
- To determine the robot's pose by the measurement of RFID tag positions, at least two tags needs to be perceived. The problem could be made simpler by considering the pose estimated using these RFID positions as measurement data rather than the measured RFID positions themselves.
- The initial state, i.e., the `estimated_base_link_pose` pose at the start of simulation is considered the same as the pose of the `odom` frame, namely `estimated_base_link_pose` can be considered as initially known with zero or constant uncertainty. Optionally, it can be updated using the `2D Pose Estimate` widget in RVIZ, in which case the mean and variance of the estimate should be reset.

### Instructions

Upload the modified `robile_navigation` package with your submission, and also paste your Kalman filter implementation in the cell below. Before starting your implementation please follow these steps:

- Modify the simulation launch file to load the `closed_walls.world` environment from the `robile_gazebo` package
- Implement pose estimation using a Kalman filter in the `robile_localisation_kalman.py` script from the `robile_navigation` repository
- Use the launch file `landmark_based_localisation.launch.py` under `robile_navigation` and modify it accordingly to launch your implementation
- Please feel free to modify any of the scripts according to your needs

### Potentially useful references

- Section 5.6.8.5 in the "Introduction to Autonomous Mobile Robots" book. It doesn't have the same setup, but is still useful
- https://www.alanzucconi.com/2022/07/24/kalman-gain/
- http://bilgin.esme.org/BitsAndBytes/KalmanFilterforDummies

In [ ]:
# YOUR CODE HERE

#!/usr/bin/env python3


import rclpy
from rclpy.node import Node
from geometry_msgs.msg import PoseStamped, PoseWithCovarianceStamped
from robile_interfaces.msg import PositionLabelled, PositionLabelledArray
from nav_msgs.msg import Odometry
import numpy as np
import tf2_ros
from tf_transformations import euler_from_quaternion, quaternion_from_euler


class LocalisationUsingKalmanFilter(Node):
    """
    Landmark based localisation using Kalman Filter
    """

    def __init__(self):
        super().__init__('localisation_using_kalman_filter')

        # declaring and getting parameters from yaml file
        self.declare_parameters(
            namespace='',
            parameters=[
                ('map_frame', 'map'),
                ('odom_frame', 'odom'),                
                ('laser_link_frame', 'base_laser_front_link'),
                ('real_base_link_frame', 'real_base_link'),
                ('scan_topic', 'scan'),
                ('odom_topic', 'odom'),
                ('rfid_tag_poses_topic', 'rfid_tag_poses'),
                ('initial_pose_topic', 'initialpose'),
                ('real_base_link_pose_topic', 'real_base_link_pose'),
                ('estimated_base_link_pose_topic', 'estimated_base_link_pose'),
                ('minimum_travel_distance', 0.1),
                ('minimum_travel_heading', 0.1),
                ('rfid_tags.A', [1.0,1.0]),
                ('rfid_tags.B', [6.0,1.0]),
                ('rfid_tags.C', [3.0,-1.0]),
                ('rfid_tags.D', [1.0,-3.0]),
                ('rfid_tags.E', [-4.0,4.0]),                        
            ])

        self.map_frame = self.get_parameter('map_frame').get_parameter_value().string_value
        self.odom_frame = self.get_parameter('odom_frame').get_parameter_value().string_value
        self.laser_link_frame = self.get_parameter('laser_link_frame').get_parameter_value().string_value
        self.real_base_link_frame = self.get_parameter('real_base_link_frame').get_parameter_value().string_value
        self.scan_topic = self.get_parameter('scan_topic').get_parameter_value().string_value
        self.odom_topic = self.get_parameter('odom_topic').get_parameter_value().string_value
        self.rfid_tag_poses_topic = self.get_parameter('rfid_tag_poses_topic').get_parameter_value().string_value
        self.initial_pose_topic = self.get_parameter('initial_pose_topic').get_parameter_value().string_value
        self.real_base_link_pose_topic = self.get_parameter('real_base_link_pose_topic').get_parameter_value().string_value
        self.estimated_base_link_pose_topic = self.get_parameter('estimated_base_link_pose_topic').get_parameter_value().string_value
        self.minimum_travel_distance = self.get_parameter('minimum_travel_distance').get_parameter_value().double_value
        self.minimum_travel_heading = self.get_parameter('minimum_travel_heading').get_parameter_value().double_value
        self.rfid_tags_A = self.get_parameter('rfid_tags.A').get_parameter_value().double_array_value
        self.rfid_tags_B = self.get_parameter('rfid_tags.B').get_parameter_value().double_array_value
        self.rfid_tags_C = self.get_parameter('rfid_tags.C').get_parameter_value().double_array_value
        self.rfid_tags_D = self.get_parameter('rfid_tags.D').get_parameter_value().double_array_value
        self.rfid_tags_E = self.get_parameter('rfid_tags.E').get_parameter_value().double_array_value

        # Create dictionary of RFID tag positions
        self.rfid_tags = {
            'A': np.array(self.rfid_tags_A),
            'B': np.array(self.rfid_tags_B),
            'C': np.array(self.rfid_tags_C),
            'D': np.array(self.rfid_tags_D),
            'E': np.array(self.rfid_tags_E)
        }

        # Initialize Kalman filter state
        # State vector [x, y, theta, v_x, v_y, omega]
        self.x = np.zeros((6, 1))
        # State covariance matrix
        self.P = np.eye(6) * 0.1
        # Process noise
        self.Q = np.eye(6) * 0.01
        # Measurement noise for RFID tags
        self.R_rfid = np.eye(2) * 0.1
        # Measurement noise for odometry
        self.R_odom = np.eye(3) * 0.05
        # Last update time
        self.last_time = self.get_clock().now().nanoseconds / 1e9
        # Last pose for calculating velocity
        self.last_pose = np.zeros(3)
        # Flag to indicate if we've received initial pose
        self.initialized = False

        # setting up subscribers
        self.odom_subscriber = self.create_subscription(
            Odometry, 
            self.odom_topic, 
            self.odom_callback, 
            10
        )
        self.rfid_tag_subscriber = self.create_subscription(
            PositionLabelledArray, 
            self.rfid_tag_poses_topic, 
            self.rfid_callback, 
            10
        )
        self.initial_pose_subscriber = self.create_subscription(
            PoseWithCovarianceStamped, 
            self.initial_pose_topic, 
            self.initial_pose_callback, 
            10
        )
        self.real_base_link_subscriber = self.create_subscription(
            PoseStamped, 
            self.real_base_link_pose_topic, 
            self.real_base_link_pose_callback, 
            10
        )
        
        # Publisher for estimated pose
        self.estimated_robot_pose_publisher = self.create_publisher(
            PoseWithCovarianceStamped, 
            self.estimated_base_link_pose_topic, 
            10
        )
        
        # setting up tf2 listener
        self.tf_buffer = tf2_ros.Buffer()
        self.tf_listener = tf2_ros.TransformListener(self.tf_buffer, self)
        
        self.get_logger().info("Kalman Filter initialized")

    def initial_pose_callback(self, msg):
        """
        Callback for initial pose estimate from RVIZ
        """
        # Extract position and orientation
        position = msg.pose.pose.position
        orientation = msg.pose.pose.orientation
        
        # Convert quaternion to Euler angles
        _, _, yaw = euler_from_quaternion([orientation.x, orientation.y, orientation.z, orientation.w])
        
        # Reset state vector with initial pose
        self.x[0, 0] = position.x
        self.x[1, 0] = position.y
        self.x[2, 0] = yaw
        self.x[3:6, 0] = 0.0  # Reset velocities
        
        # Reset covariance matrix
        self.P = np.eye(6) * 0.1
        
        # Reset last pose
        self.last_pose = np.array([position.x, position.y, yaw])
        
        # Reset last time
        self.last_time = self.get_clock().now().nanoseconds / 1e9
        
        # Set initialized flag
        self.initialized = True
        
        self.get_logger().info(f"Initial pose set: x={position.x:.2f}, y={position.y:.2f}, theta={yaw:.2f}")
        
        # Publish initial estimated pose
        self.publish_estimated_pose()

    def real_base_link_pose_callback(self, msg):
        """
        Updating the base_link pose based on the update in robile_rfid_tag_finder.py
        """
        # This is just for visualization/debugging, not used in the filter
        yaw = euler_from_quaternion([msg.pose.orientation.x, msg.pose.orientation.y, msg.pose.orientation.z, msg.pose.orientation.w])[2]
        self.real_laser_link_pose = [msg.pose.position.x, msg.pose.position.y, yaw]
        self.get_logger().debug(f"Real base link pose: x={msg.pose.position.x:.2f}, y={msg.pose.position.y:.2f}, theta={yaw:.2f}")

    def predict(self, dt):
        """Prediction step of the Kalman filter"""
        # State transition matrix
        F = np.eye(6)
        F[0, 3] = dt  # x += v_x * dt
        F[1, 4] = dt  # y += v_y * dt
        F[2, 5] = dt  # theta += omega * dt
        
        # Predict state
        self.x = F @ self.x
        # Predict covariance
        self.P = F @ self.P @ F.T + self.Q
        
        # Log prediction step output
        self.get_logger().info(f"KF Predict: x={self.x[0,0]:.2f}, y={self.x[1,0]:.2f}, theta={self.x[2,0]:.2f}, P_diag={np.diag(self.P).tolist()}")


    def update_with_odom(self, odom_measurement):
        """Update step with odometry measurements"""
        # Measurement matrix for odometry (x, y, theta)
        H = np.zeros((3, 6))
        H[0, 0] = 1  # x
        H[1, 1] = 1  # y
        H[2, 2] = 1  # theta
        
        # Ensure odom_measurement is a column vector (3,1)
        # This explicitly reshapes it, which can prevent subtle broadcasting issues
        # that might lead to unexpected dimensions in `y` later.
        odom_measurement_col_vector = odom_measurement.reshape(-1, 1)

        # Innovation (measurement residual)
        # Use the explicitly reshaped column vector for `odom_measurement`
        y = odom_measurement_col_vector - H @ self.x
        
        # Normalize theta difference to [-pi, pi]
        y[2] = np.arctan2(np.sin(y[2]), np.cos(y[2]))
        
        # Reshape y to be a column vector (3,1) - this line becomes technically redundant but harmless
        # if odom_measurement_col_vector was already (3,1)
        y = y.reshape(-1, 1) 
        
        # Innovation covariance
        S = H @ self.P @ H.T + self.R_odom
        
        # Kalman gain
        K = self.P @ H.T @ np.linalg.inv(S)
        
        # Update state
        self.x = self.x + K @ y
        
        # Update covariance
        self.P = (np.eye(6) - K @ H) @ self.P
        
        # Log odometry update output
        self.get_logger().info(f"KF Update Odom: x={self.x[0,0]:.2f}, y={self.x[1,0]:.2f}, theta={self.x[2,0]:.2f}, P_diag={np.diag(self.P).tolist()}")


    def update_with_rfid(self, rfid_measurements):
        """Update step with RFID measurements"""
        if not rfid_measurements:
            return
            
        for tag_id, measurement in rfid_measurements.items():
            if tag_id not in self.rfid_tags:
                continue
                
            # Get the true position of the RFID tag
            tag_pos = self.rfid_tags[tag_id]
            
            # Measurement function (nonlinear)
            # Expected measurement based on current state
            dx = tag_pos[0] - self.x[0, 0]
            dy = tag_pos[1] - self.x[1, 0]
            expected_range = np.sqrt(dx**2 + dy**2)
            expected_bearing = np.arctan2(dy, dx) - self.x[2, 0]
            expected_measurement = np.array([expected_range, expected_bearing])
            
            # Actual measurement
            actual_measurement = np.array(measurement)
            
            # Innovation (measurement residual)
            y = actual_measurement - expected_measurement
            
            # Normalize bearing difference to [-pi, pi]
            y[1] = np.arctan2(np.sin(y[1]), np.cos(y[1]))
            
            # Reshape y to be a column vector (2,1)
            y = y.reshape(-1, 1)
            
            # Jacobian of measurement function
            H = np.zeros((2, 6))
            H[0, 0] = -dx / expected_range
            H[0, 1] = -dy / expected_range
            H[1, 0] = dy / (dx**2 + dy**2)
            H[1, 1] = -dx / (dx**2 + dy**2)
            H[1, 2] = -1
            
            # Innovation covariance
            S = H @ self.P @ H.T + self.R_rfid
            
            # Kalman gain
            K = self.P @ H.T @ np.linalg.inv(S)
            
            # Update state
            self.x = self.x + K @ y
            
            # Update covariance
            self.P = (np.eye(6) - K @ H) @ self.P
            
            # Log RFID update output
            self.get_logger().info(f"KF Update RFID ({tag_id}): x={self.x[0,0]:.2f}, y={self.x[1,0]:.2f}, theta={self.x[2,0]:.2f}, P_diag={np.diag(self.P).tolist()}")


    def odom_callback(self, msg):
        """
        Callback for odometry messages
        """
        if not self.initialized:
            # Initialize with first odometry message
            position = msg.pose.pose.position
            orientation = msg.pose.pose.orientation
            _, _, yaw = euler_from_quaternion([orientation.x, orientation.y, orientation.z, orientation.w])
            
            self.x[0, 0] = position.x
            self.x[1, 0] = position.y
            self.x[2, 0] = yaw
            self.x[3:6, 0] = 0.0 # Initialize velocities to zero
            self.last_pose = np.array([position.x, position.y, yaw])
            self.last_time = self.get_clock().now().nanoseconds / 1e9
            self.initialized = True
            self.get_logger().info(f"Initialized with odometry: x={position.x:.2f}, y={position.y:.2f}, theta={yaw:.2f}")
            self.publish_estimated_pose()
            return
        
        # Extract position and orientation
        position = msg.pose.pose.position
        orientation = msg.pose.pose.orientation
        
        # Convert quaternion to Euler angles
        _, _, yaw = euler_from_quaternion([orientation.x, orientation.y, orientation.z, orientation.w])
        
        # Create measurement vector [x, y, theta] as a column vector
        # This explicitly makes it a (3,1) array
        odom_measurement = np.array([[position.x], [position.y], [yaw]])
        
        # Get current time and calculate dt
        current_time = self.get_clock().now().nanoseconds / 1e9
        dt = current_time - self.last_time
        
        self.get_logger().info(f"Odom Callback: dt={dt:.4f}") # Added log for dt

        if dt > 0:
            # Calculate velocities
            # Using current odometry position, not filtered state, for velocity calculation
            dx = (position.x - self.last_pose[0]) / dt
            dy = (position.y - self.last_pose[1]) / dt
            # Normalize dtheta before dividing by dt
            dtheta = np.arctan2(np.sin(yaw - self.last_pose[2]), np.cos(yaw - self.last_pose[2])) / dt
            
            # Update velocity estimates in state with a low-pass filter
            # These values will then be used in the next prediction step
            self.x[3, 0] = 0.7 * self.x[3, 0] + 0.3 * dx  
            self.x[4, 0] = 0.7 * self.x[4, 0] + 0.3 * dy
            self.x[5, 0] = 0.7 * self.x[5, 0] + 0.3 * dtheta
            
            # Predict step
            self.predict(dt)
            
            # Update step with odometry
            self.update_with_odom(odom_measurement)
            
            # Update last pose and time for next velocity calculation
            self.last_pose = np.array([position.x, position.y, yaw])
            self.last_time = current_time
            
            # Publish estimated pose
            self.publish_estimated_pose()
        else:
            self.get_logger().warn(f"dt is zero or negative ({dt:.4f}), skipping prediction and update.")


    def rfid_callback(self, msg):
        """
        Based on the detected RFID tags, performing measurement update
        """
        if not self.initialized:
            self.get_logger().warn("RFID callback received but Kalman filter not initialized.")
            return
            
        # Process RFID tag detections
        rfid_measurements = {}
        
        for tag in msg.positions:
            # Extract tag ID using the correct attribute name
            tag_id = tag.name
                
            # Extract position (relative to laser_link_frame as per your setup)
            position = tag.position
            
            # Calculate range and bearing to the tag from the sensor's perspective
            range_to_tag = np.sqrt(position.x**2 + position.y**2)
            bearing_to_tag = np.arctan2(position.y, position.x)
            
            # Store measurement
            rfid_measurements[tag_id] = [range_to_tag, bearing_to_tag]
            
            self.get_logger().debug(f"Detected RFID tag {tag_id} at range={range_to_tag:.2f}, bearing={bearing_to_tag:.2f}")
        
        # Update Kalman filter with RFID measurements
        if rfid_measurements:
            self.update_with_rfid(rfid_measurements)
            self.publish_estimated_pose()
        else:
            self.get_logger().debug("No RFID measurements to process.")


    def publish_estimated_pose(self):
        """
        Publish the current estimated pose
        """
        # Create PoseWithCovarianceStamped message
        pose_msg = PoseWithCovarianceStamped()
        pose_msg.header.stamp = self.get_clock().now().to_msg()
        pose_msg.header.frame_id = self.map_frame
        
        # Set position
        pose_msg.pose.pose.position.x = self.x[0, 0]
        pose_msg.pose.pose.position.y = self.x[1, 0]
        pose_msg.pose.pose.position.z = 0.0
        
        # Set orientation (convert theta to quaternion)
        q = quaternion_from_euler(0, 0, self.x[2, 0])
        pose_msg.pose.pose.orientation.x = q[0]
        pose_msg.pose.pose.orientation.y = q[1]
        pose_msg.pose.pose.orientation.z = q[2]
        pose_msg.pose.pose.orientation.w = q[3]
        
        # Set covariance (6x6 matrix in row-major order)
        # Only set position and orientation covariance relevant for PoseWithCovarianceStamped
        # Note: The covariance matrix self.P is 6x6, covering [x, y, theta, vx, vy, omega]
        # The PoseWithCovarianceStamped message expects a 6x6 covariance for [x, y, z, roll, pitch, yaw]
        # We map x, y, yaw from our 6x6 P to the corresponding spots.
        covariance = np.zeros(36)
        covariance[0] = self.P[0, 0]   # Variance of x
        covariance[1] = self.P[0, 1]   # Covariance of x and y
        covariance[5] = self.P[0, 2]   # Covariance of x and theta
        
        covariance[6] = self.P[1, 0]   # Covariance of y and x
        covariance[7] = self.P[1, 1]   # Variance of y
        covariance[11] = self.P[1, 2]  # Covariance of y and theta
        
        covariance[30] = self.P[2, 0]  # Covariance of theta and x
        covariance[31] = self.P[2, 1]  # Covariance of theta and y
        covariance[35] = self.P[2, 2]  # Variance of theta
        
        pose_msg.pose.covariance = covariance.tolist()
        
        # Publish the pose
        self.estimated_robot_pose_publisher.publish(pose_msg)
        
        self.get_logger().info(f"Published estimated pose: x={self.x[0,0]:.2f}, y={self.x[1,0]:.2f}, theta={self.x[2,0]:.2f}")


def main(args=None):
    rclpy.init(args=args)

    try:
        localisation_using_kalman_filter = LocalisationUsingKalmanFilter()
        rclpy.spin(localisation_using_kalman_filter)

    finally:
        localisation_using_kalman_filter.destroy_node()
        rclpy.shutdown()

if __name__ == '__main__':
    main()


Finally, comment on your observations while evaluating your implementation in the cell below, and include relevant screenshots to verify your implementation.

YOUR ANSWER HERE

1. We were able to implement the kalman filter and also visualize it on Rviz.
2. The kalman filter is basically taking the input and denoising the signal and producing a smooth transition between the different poses.
3. The kalman filter is always following the pose direction but its transitions are smoother.

Video of implementation:

https://drive.google.com/file/d/1iPsIhFSOqTgQiYx8wNW_6IhWnH5Wqb0m/view?usp=drive_link